In [1]:
from google.colab import files
files.upload()

Saving customer_queries.csv to customer_queries.csv


{'customer_queries.csv': b',question_text,question_topic\r\n0,"Hi! If I sign up for your email list, can I select to get emails exclusively for sale products? I\'m really only interested in shopping clearance deals.",Sales/Promotions\r\n1,"I\'m going to be out of the country for about a week, but before I travel I\'m going to be getting the animal print jumper, product code T383434J. Can I have this held at the post office or a distribution centre for pickup? I\'d just hate to have it sitting at my house when I\'m not going to be around.",Shipping\r\n2,"I was wondering if you\'d be able to overnight a jacket (Item #5220032) to me in Trenton, NJ?",Shipping\r\n3,"The Swingline electronic stapler (472555) looks really great, but I would need to have it shipped to St. Thomas in the US Virgin Islands. Do you allow for that, or is that considered ""international"" shipping?",Shipping\r\n4,"I think this cosmetic bag would work great for me! However, do you know how long it would take to send 

In [1]:
!pip install pydot
!pip install tensorboardcolab

from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import preprocessing
import numpy as np


Using TensorFlow backend.


In [3]:
#tensorboard view
from tensorboardcolab import *
tbc = TensorBoardColab()
summary_writer = tbc.get_writer()

Wait for 8 seconds...
TensorBoard link:
http://98181273.ngrok.io


In [4]:
# loading data
data = pd.read_csv('customer_queries.csv')
data.drop(columns = ['Unnamed: 0'],inplace = True)
data.head(5)

,question_text,question_topic
0,"Hi! If I sign up for your email list, can I se...",Sales/Promotions
1,I'm going to be out of the country for about a...,Shipping
2,I was wondering if you'd be able to overnight ...,Shipping
3,The Swingline electronic stapler (472555) look...,Shipping
4,I think this cosmetic bag would work great for...,Shipping


In [5]:
# unique labels
data['question_topic'].unique()

array(['Sales/Promotions', 'Shipping', 'Product Availability',
       'Product Specifications', 'Omnichannel', 'Product Comparison',
       'Returns & Refunds'], dtype=object)

In [0]:
# assiging X,y and label encoding
X = data['question_text'].copy()
y = data['question_topic'].copy()
le = preprocessing.LabelEncoder()
le.fit(y)
l = le.transform(y) 

In [0]:
# splitting data into train and test set
x_train,x_test,y_train,y_test = train_test_split(X,l,test_size = 0.3,random_state=42)

In [0]:
# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

In [0]:
# calculate the maximum document length
def max_length(lines):
	return max([len(s.split()) for s in lines])

In [0]:
# encode a list of lines
def encode_text(tokenizer, lines, length):
	# integer encode
	encoded = tokenizer.texts_to_sequences(lines)
	# pad encoded sequences
	padded = pad_sequences(encoded, maxlen=length, padding='post')
	return padded

In [0]:
# define the model
def define_model(length, vocab_size):
	# channel 1
	inputs1 = Input(shape=(length,))
	embedding1 = Embedding(vocab_size, 100)(inputs1)
	conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
	drop1 = Dropout(0.5)(conv1)
	pool1 = MaxPooling1D(pool_size=2)(drop1)
	flat1 = Flatten()(pool1)
	# channel 2
	inputs2 = Input(shape=(length,))
	embedding2 = Embedding(vocab_size, 100)(inputs2)
	conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
	drop2 = Dropout(0.5)(conv2)
	pool2 = MaxPooling1D(pool_size=2)(drop2)
	flat2 = Flatten()(pool2)
	# channel 3
	inputs3 = Input(shape=(length,))
	embedding3 = Embedding(vocab_size, 100)(inputs3)
	conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
	drop3 = Dropout(0.5)(conv3)
	pool3 = MaxPooling1D(pool_size=2)(drop3)
	flat3 = Flatten()(pool3)
	# merge
	merged = concatenate([flat1, flat2, flat3])
	# interpretation
	dense1 = Dense(10, activation='relu')(merged)
	outputs = Dense(7, activation='softmax')(dense1)
	model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
	# compile
	model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model


In [12]:
# training dataset % testing dataset
trainLines, trainLabels = x_train,y_train
testLines, testLabels = x_test,y_test
# create tokenizer
tokenizer = create_tokenizer(trainLines)
# calculate max document length
length = max_length(trainLines)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, trainLines, length)
print(trainX.shape)
print(trainX)
# define model
model = define_model(length, vocab_size)
# fit model
model.fit([trainX,trainX,trainX], array(trainLabels), epochs=10, batch_size=16,callbacks=[TensorBoardColabCallback(tbc)])

Max document length: 86
Vocabulary size: 7530
(3500, 86)
[[  8   3  27 ...   0   0   0]
 [ 17 331  40 ...   0   0   0]
 [  3  68   3 ...   0   0   0]
 ...
 [  6   4   7 ...   0   0   0]
 [  3  52   2 ...   0   0   0]
 [ 11  72 118 ...   0   0   0]]
Epoch 1/10
3500/3500 [==============================] - 15s 4ms/step - loss: 1.3450 - acc: 0.4211
Epoch 2/10
3500/3500 [==============================] - 14s 4ms/step - loss: 0.3377 - acc: 0.8906
Epoch 3/10
3500/3500 [==============================] - 14s 4ms/step - loss: 0.0574 - acc: 0.9911
Epoch 4/10
3500/3500 [==============================] - 14s 4ms/step - loss: 0.0198 - acc: 0.9954
Epoch 5/10
3500/3500 [==============================] - 14s 4ms/step - loss: 0.0094 - acc: 0.9989
Epoch 6/10
3500/3500 [==============================] - 14s 4ms/step - loss: 0.0078 - acc: 0.9974
Epoch 7/10
3500/3500 [==============================] - 14s 4ms/step - loss: 0.0091 - acc: 0.9991
Epoch 8/10
3500/3500 [==============================] - 14s 4ms/s

In [13]:
# create tokenizer
tokenizer = create_tokenizer(trainLines)
# calculate max document length
length = max_length(trainLines)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, trainLines, length)
testX = encode_text(tokenizer, testLines, length)
print(trainX.shape, testX.shape)
 
# evaluate model on training dataset
loss, acc = model.evaluate([trainX,trainX,trainX], array(trainLabels), verbose=0)
print('Train Accuracy: %f' % (acc*100))
 
# evaluate model on test dataset dataset
loss, acc = model.evaluate([testX,testX,testX],array(testLabels), verbose=0)
print('Test Accuracy: %f' % (acc*100))

Max document length: 86
Vocabulary size: 7530
(3500, 86) (1500, 86)
Train Accuracy: 99.828571
Test Accuracy: 94.200000


In [14]:
# input review
rev = input()
s  = [rev]
t = create_tokenizer(s)
review = encode_text(tokenizer, s, length)

when is the new sales coming out


In [15]:
# encode
review

array([[ 35,  16,   1, 173, 189, 542, 120,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0]], dtype=int32)

In [16]:
# prediction 
answer = model.predict([review,review,review], batch_size=None, verbose=0, steps=None)
ind = np.argmax(answer)
ansarr = le.classes_
print(ansarr[ind])

Sales/Promotions


In [17]:
# probability
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})
answer

array([[ 0.0486,  0.0745,  0.0029,  0.1259,  0.0003,  0.7476,  0.0001]],
      dtype=float32)

In [18]:
le.classes_

array(['Omnichannel', 'Product Availability', 'Product Comparison',
       'Product Specifications', 'Returns & Refunds', 'Sales/Promotions',
       'Shipping'], dtype=object)